In [1]:
import numpy as np
import control
from misc_tools.print_latex import print_tex

input example : 
>>> arr_T = np.array([[r'\vec{v}_1', r'\vec{v}_2']]).T
>>> print_tex(arr_T,'=', np.arange(1,5).reshape(2,-1)/4, r'; symbols: \otimes, \cdot,\times')
output: 


<IPython.core.display.Math object>

# Pole placement for control system

# Sources

(Matlab) Pole Placement | State Space, Part 2: https://se.mathworks.com/videos/state-space-part-2-pole-placement-1547198830727.html

https://python-control.readthedocs.io/en/latest/generated/control.place.html


## Check stability

Lets consider a system
$$ \dot x = A x + B u$$
$$\begin{bmatrix}
\dot x_1 \\ \dot x_2
\end{bmatrix}=
\begin{bmatrix}
0 & 1 \\ 2 & -1
\end{bmatrix}
\begin{bmatrix}
 x_1 \\  x_2
\end{bmatrix}
+
\begin{bmatrix}
 1 \\  0
\end{bmatrix}
\vec u
$$
Eigenvalues of matrix 
$$A=\begin{bmatrix}
0 & 1 \\ 2 & -1
\end{bmatrix}$$
can be found via characteristic equation
$$\det \left(\begin{bmatrix}
-\lambda & 1 \\ 2 & -1-\lambda
\end{bmatrix}\right) = 0$$

$$\lambda(1+\lambda)-2 = \lambda^2 + \lambda - 2 = 0$$
$$ \lambda_{1,2} = \frac{-1 \pm \sqrt{1^2 + 4\cdot 2}}{2} = \frac{-1 \pm 3}{2} = \{1, -2\}$$
We have system with one positive eigenvalue thus, by itself, it is unstable.

In [2]:
A = np.array([  [0, 1],
                [2,-1]])
print_tex(np.linalg.eigvals(A))

<IPython.core.display.Math object>

# Force stability via control

Suppose that our control has two "knobs" $k_1$ and $k_1$, and we use linear/proportional control
$$\vec u = - K \vec x$$
new system can be written as
$$ \dot{ \vec x } = \left(A - BK\right) \vec x = A_{CL} \ \vec x$$ 
$$A_{CL}= 
\begin{bmatrix}
0 & 1 \\ 2 & -1
\end{bmatrix}
- 
\begin{bmatrix}
 1 \\  0
\end{bmatrix}
K$$

$BK$ should have dimensions $2 \times 2$, so appropriate choice of $K$ is
$$K = \vec k^T = 
\begin{bmatrix}
 k_1 & k_2
\end{bmatrix}$$

$$BK = \begin{bmatrix}
1 \\  0
\end{bmatrix}
\begin{bmatrix}
 k_1 & k_2
\end{bmatrix}=
\begin{bmatrix}
k_1 & k_2 \\ 0 & 0
\end{bmatrix}
$$

$$A_{CL}= 
\begin{bmatrix}
0 & 1 \\ 2 & -1
\end{bmatrix}-
\begin{bmatrix}
k_1 & k_2 \\ 0 & 0
\end{bmatrix}=
\begin{bmatrix}
-k_1 & 1-k_2 \\ 2 & -1
\end{bmatrix}$$

Characteristic equation for controlled system is



$$(-k_1 - \lambda)(-1 -\lambda)-2(1-k_2) = 0 $$
$$(k_1 + \lambda)(1 +\lambda)+2(1-k_2) = 0 $$
$$\lambda^2 + (1+k_1)\lambda + (2 +k_1 - 2k_2) = 0 $$
naturally, if $k_1 = k_2 = 0$, we retrieve equation for unstable system
$$\lambda^2 + \lambda - 2 = 0$$

Suppose we want to choose such $k_1$ and $k_1$, that we get a system with negative eigenvalues 
$$\lambda_{1,2} = \{-1,-2\}$$
It should have a characteristic equation
$$(\lambda + 1)(\lambda + 2) = 0$$
$$ \lambda^2 + 3\lambda + 2 = 0$$

So we can select "knob" values so following holds
$$\lambda^2 + (1+k_1)\lambda + (2 +k_1 - 2k_2) = \lambda^2 + 3\lambda + 2$$
which would mean that
$$\begin{cases}
1+k_1 = 3\\
2 +k_1 - 2k_2 = 2
\end{cases}
\longrightarrow
\begin{cases}
k_1 = 2 \\
k_2 = 1
\end{cases}$$

$$K = \vec k^T = 
\begin{bmatrix}
2 & 1
\end{bmatrix}$$

In [3]:
B = np.array([[1,0]]).T
K = np.array([[2,1]])
A_cl = A - B @ K
eigs = np.linalg.eigvals(A_cl)
print_tex("A_{CL} = ", A_cl, "\ ; \\lambda_{1,2}= ", eigs)

<IPython.core.display.Math object>

Or we can use control library and "place" method

In [7]:
eigs_set = np.array([-1,-2])
K2 = control.place(A,B, eigs_set)
print_tex("K = ", K2, r"\ ; \vec \lambda= ", np.linalg.eigvals(A - B @ K2))

<IPython.core.display.Math object>